In [429]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
import re
from datasets import load_dataset
from datasets import Dataset
from tqdm.notebook import tqdm
from transformers import AutoTokenizer

In [545]:
raw_datasets  = load_dataset("conll2003")
model_checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
df_errorIds = pd.read_csv("Error_Ids.csv")
df_errorIds = df_errorIds.drop(['Unnamed: 0'], axis = 1)

tokenDict = {
    "Lead" : 0,
    "Position" : 1,
    "Evidence" : 2,
    "Claim" : 3,
    "Concluding Statement" : 4,
    "Counterclaim" : 5,
    "Rebuttal": 6
}

100%|██████████| 3/3 [00:00<00:00, 107.43it/s]


In [546]:
# helper functions

def fileToArray(file_path):
    # turn text into array of words
    text_ds = tf.data.TextLineDataset(file_path).filter(lambda x: tf.cast(tf.strings.length(x), bool))
    text_ds = text_ds.enumerate()
    docTxt = []
    for i in text_ds.as_numpy_iterator():
        line = i[1].decode().split()
        for l in line:
            docTxt.append(l)
    
    docTxt = ' '.join(docTxt)
    return docTxt

def calc_word_indices(full_text, discourse_start, discourse_end):
    start_index = len(full_text[:discourse_start].split())
    token_len = len(full_text[discourse_start:discourse_end].split())
    output = list(range(start_index, start_index + token_len))
    if output[-1] >= len(full_text.split()):
        output = list(range(start_index, start_index + token_len-1))
    return output

def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )

    all_labels = examples["ner_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

def dfTags_to_ints(df):
    for index, row in df.iterrows():
        intArray = []
        tokenArray = []
        for i in row[2]:
            intArray.append(int(i))
        df.at[index, "ner_tags"] = intArray.copy()
        # for j in row[1]:
        #     if j != '.' and j != ',':
        #         tokenArray.append(j)
        # df.at[index, "tokens"] = tokenArray.copy()
    return df

In [543]:
#takes in a csv file name and returns a tokenized dataset
#NEEDS to be updated in occurance with the createTokenTrainingSet() function
def createTokenDataset(fileName):

    #maybe create a helper function???
    df = csv_to_df(fileName)
    essayNames = df["id"].unique()
    
    data = {
        "id": [],
        "tokens": [],
        "ner_tags": []
    }

    token_df = pd.DataFrame(data)
    for fileName in tqdm(essayNames):
        df_file = df[df.id.str.contains(fileName,case=False)]
        df_file = df_file.reset_index()
        labels = []
        num_labels = []
        fullText = []
        for i in range(len(df_file.index)):
            df_string = df_file["predictionstring"][i]
            stringArray = df_string.split()
            df_textString = df_file["discourse_text"][i].split()
            for df in df_textString:
                if df != '.' and df != ',':
                    fullText.append(df)
            for s in stringArray:
                labels.append(df_file["discourse_type"][i])
                num_labels.append(tokenDict[str(df_file["discourse_type"][i])])

        token_df.loc[len(token_df.index)] = [fileName, fullText, num_labels]
    
    #find a better way to do this
    token_df['tokens'] = token_df['tokens'].apply(lambda a: ' '.join(map(str, a)))
    token_df['ner_tags'] = token_df['ner_tags'].apply(lambda a: ' '.join(map(str, a)))
    token_df['tokens'] = token_df['tokens'].apply(lambda a: a.split())
    token_df['ner_tags'] = token_df['ner_tags'].apply(lambda a: a.split())

    tokenDataset = Dataset.from_pandas(token_df)

    tokenized_datasets = tokenDataset.map(
        tokenize_and_align_labels,
        batched=True,
    )
    return tokenized_datasets

def createTokenTrainingSet(fileName):
    errorList = list(df_errorIds["id"])
    df = pd.read_csv(fileName)

    df['tokens'] = df['tokens'].apply(lambda a: a.split())
    df['ner_tags'] = df['ner_tags'].apply(lambda a: a.split())
    df = df.drop(['Unnamed: 0'], axis = 1)

    errorList_idx = list(map(lambda e: df.loc[df["id"] == e].index[0], errorList))
    df = df.drop(errorList_idx, 0)
    df = dfTags_to_ints(df)

    newDataset = Dataset.from_pandas(df)

    tokenized_datasets = newDataset.map(
        tokenize_and_align_labels,
        batched=True,
    )
    return tokenized_datasets

    

In [547]:
tokenTrain = createTokenTrainingSet("trainHugging4.csv")

 12%|█▎        | 2/16 [00:06<00:47,  3.41s/ba]


KeyboardInterrupt: 